# Creating fake data for analysis

This notebook illustrates the methods behind the dataset we use in our analysis.


Goals:
1. Create a dataset similar in shape and content to that of COMIS (?)
  - one row for each course taken
      - student features
        - ids/ssn
        - demographics (age, sex)
        - student intent (transfer, AA, credential)
        - first time or returning
      - course features
        - course id
        - section id
        - units
        - grade in class
        - college id
        - department (Math, English)
        - dev ed (T/F)
      - term id (Year, Term)
2. Prepare data in similar fashion to that we receive
  - categoricals
  - fall-winter-spring-quarter system
3. Prepare a second dataset for __Enrollment__ (10 million rows)
  - columns: term, course name, course code, college id, units, credit/noncredit 

***
__Set up__

In [1]:
import pandas as pd
import numpy as np

We will end up with a dataset 10 million rows long but it should only have 1 million students (taking at most 10 classes).

First, the students: <br>
They must have 
1. id/ssn 
2. age
3. sex 
4. intent 
5. first time status 
6. full/part-time status

In [2]:
np.random.seed(41)

intent = np.random.choice(a = ['transfer', 'AA', 'credential'], size = 1_000_000, p = [.6, .2, .2])

sex = np.random.choice(a = ['male', 'female'], size = 1_000_000, p = [.49, .51])
first_time = np.random.choice(a = ['first', 'returning'], size = 1_000_000, p = [.3, .7])

# the following will return ~one million different 'ids' but not exactly one million if some numbers are repeated by chance
# that is ok though
ids = np.random.randint(low = 100_000, high = 999_999, size = 1_000_000,) 

age = np.random.randint(low = 18, high = 65, size = 1_000_000,)

race_ethnicity = np.random.choice(a = ['asian', 'white', 'latino', 'black', 'other'], size = 1_000_000, p = [0.15, 0.35, 0.38, 0.10, 0.02])

full_part = np.random.choice(a = ['full-time', 'part-time'], size = 1_000_000, p = [0.3158, 0.6842]) # 2013 stats according to IPEDS 

In [3]:
data = pd.DataFrame(
    data = 
    {
        'ids': ids, 
        'sex': sex, 
        'age': age, 
        'first_time_status': first_time, 
        'intent': intent, 
        'race': race_ethnicity,
        'full_time': full_part,
    }, 
)

data.head()

,ids,sex,age,first_time_status,intent,race,full_time
0,677074,female,18,first,transfer,white,part-time
1,344008,female,50,first,transfer,asian,part-time
2,165437,male,18,returning,AA,latino,part-time
3,501411,male,52,returning,transfer,other,full-time
4,983007,male,21,returning,transfer,white,part-time


In [4]:
print(f"Only {data['ids'].nunique() / len(data['ids']):.2%} of ids are unique. Meaning, we have {data['ids'].nunique():,.0f} students in total.")

Only 60.36% of ids are unique. Meaning, we have 603,584 students in total.


We need to then delete those extra instances of the ID's (de-duplicate them).

In [5]:
data.drop_duplicates(subset = 'ids', keep = 'first', inplace = True)

### TODO:
1. create class course's data 
2. merge both datasets into 1 big ~10 mill dataset.

IDEA:
Create a 10 million row series choosing from `data['ids'].unique()` and then `pd.merge(10mil_ids, data)` then then add course features. 

In [6]:
big_data = np.random.choice(a = data['ids'].unique(), size = 10_000_000)

big_data = pd.DataFrame(big_data)
big_data.columns = ['ids']

big_data = pd.merge(big_data, data, how = 'left')

print(big_data.shape)
print(big_data.head())

(10000000, 7)
      ids     sex  age first_time_status      intent    race  full_time
0  471900  female   38         returning          AA   asian  full-time
1  506657  female   35         returning    transfer   asian  full-time
2  780835  female   58         returning  credential  latino  part-time
3  258697  female   59         returning    transfer   white  full-time
4  677933  female   45         returning    transfer  latino  full-time


In [7]:
# delete unnecessary data as we go
del data
del ids
del sex
del age
del first_time
del intent
del race_ethnicity
del full_part

### Adding CCC

source to this table: https://en.wikipedia.org/wiki/List_of_California_Community_Colleges_by_enrollment

In [8]:
ccc = pd.read_csv("../data/raw/list_CCC.csv")

ccc.head()

,Ranking,College,Total enrollment,Full-time enrollment,Part-time enrollment
0,1,East Los Angeles College,36606,7090,29516
1,2,Santa Monica College,29999,10720,19279
2,3,American River College,29701,7560,22141
3,4,Santa Ana College,28598,3435,25163
4,5,Mount San Antonio College,28481,10499,17982


In [9]:
ccc['share_of_pop'] = ccc['Total enrollment'] / ccc['Total enrollment'].sum()

ccc.head()

,Ranking,College,Total enrollment,Full-time enrollment,Part-time enrollment,share_of_pop
0,1,East Los Angeles College,36606,7090,29516,0.024851
1,2,Santa Monica College,29999,10720,19279,0.020366
2,3,American River College,29701,7560,22141,0.020164
3,4,Santa Ana College,28598,3435,25163,0.019415
4,5,Mount San Antonio College,28481,10499,17982,0.019335


In [10]:
college = np.random.choice(a = ccc['College'], size = 10_000_000, p = list(ccc['share_of_pop']))

big_data['college'] = college

big_data.head()

,ids,sex,age,first_time_status,intent,race,full_time,college
0,471900,female,38,returning,AA,asian,full-time,Butte College
1,506657,female,35,returning,transfer,asian,full-time,San Diego Mesa College
2,780835,female,58,returning,credential,latino,part-time,Sierra College
3,258697,female,59,returning,transfer,white,full-time,De Anza College
4,677933,female,45,returning,transfer,latino,full-time,San Jose City College


In [11]:
# delete unnecessary data as we go
del ccc
del college

### Courses

Each course has 
1. top code
2. subject
3. course number
4. course name
5. units

To make sure that even when we randomize courses for the 10 million rows these 5 attributes match each other we'll save the course in one string as: `TOPCODE_SUBJECT_COURSENUMBER_COURSENAME_UNITS` and use the `.str.split("_", expand = True)` method after randomizing.

In [12]:
# Example
course_example = np.array(['123456_ENG_101_Intro to Composition_4', '123456_ENG_102_Intermediate Composition_4', '123456_ENG_103_Advanced Composition_4', '123456_ENG_99_Dev English_4'])

course_df = pd.DataFrame(course_example, columns = ['course_all'])
course_df.head()

,course_all
0,123456_ENG_101_Intro to Composition_4
1,123456_ENG_102_Intermediate Composition_4
2,123456_ENG_103_Advanced Composition_4
3,123456_ENG_99_Dev English_4


In [13]:
course_df['course_all'].str.split("_", expand = True,)

,0,1,2,3,4
0,123456,ENG,101,Intro to Composition,4
1,123456,ENG,102,Intermediate Composition,4
2,123456,ENG,103,Advanced Composition,4
3,123456,ENG,99,Dev English,4


We'll concentrate on 6 subjects: 

| SUBJECT | TOPCODE |
|:-------:|:-------:|
| Math (MAT) | 000102 |
| English (ENG) | 000104 |
| History (HIS) | 000156 |
| Statistics (STA) | 004028 |
| Economics (ECN) | 000122 |
| English as a Second Language (ESL) | 000322 |

And add at least 4 courses to each

In [14]:
math = [
    "000102_MAT_98_Pre-pre-Algebra_2",
    "000102_MAT_99_Pre-Algebra_2",
    "000102_MAT_100_Algebra_4",
    "000102_MAT_102_Linear Algebra_4",
    "000102_MAT_103_Linear Algebra II_4",
    "000102_MAT_110_Calculus_4",
    "000102_MAT_105_Geometry_4",
    "000102_MAT_107_Trigonometry_4",
]

In [15]:
english = [
    "000104_ENG_96_Pre-pre-pre-pre-College Composition_2",
    "000104_ENG_97_Pre-pre-pre-College Composition_2",
    "000104_ENG_98_Pre-pre-College Composition_2",
    "000104_ENG_99_Pre-College Composition_2",
    "000104_ENG_100_College Composition_4",
    "000104_ENG_101_College Composition II_4",
    "000104_ENG_102_American Literature_2",
    "000104_ENG_15_Reading I_2",
    "000104_ENG_25_Reading II_2",
    "000104_ENG_35_Reading III_2",
    "000104_ENG_45_Reading IIII_2",
    "000104_ENG_55_Reading V_2",
]

In [16]:
history = [
    "000156_HIS_100_World History I_3",
    "000156_HIS_101_World History II_3",
    "000156_HIS_110_Western Civilizations_4",
    "000156_HIS_115_Eastern Civilizations_5",
    "000156_HIS_120_Intro to Colonization_4",
    "000156_HIS_121_Colonization: Europe_5",
    "000156_HIS_130_The Industrial Revolution_4",
]

In [17]:
statistics = [
    "004028_STA_60_Pre-Statitics_3",
    "004028_STA_70_Statitics_3",
    "004028_STA_75_Business Statitics_3",
    "004028_STA_77_Bio Statitics_3",
]

In [18]:
economics = [
    "000122_ECN_100_Intro to Micro Economics_4",
    "000122_ECN_101_Intro to Macro Economics_4",
    "000122_ECN_110_Micro Economics_4",
    "000122_ECN_111_Macro Economics_4",
    "000122_ECN_120_Developmental Economics_4",
    "000122_ECN_130_Labor Economics_4",
    "000122_ECN_140_Health Economics_4",
]

In [19]:
esl = [
    "000322_ESL_10_Writing I_3",
    "000322_ESL_11_Reading I_3",
    "000322_ESL_12_Speaking I_2",
    "000322_ESL_20_Writing II_3",
    "000322_ESL_21_Reading II_3",
    "000322_ESL_22_Speaking II_2",
    "000322_ESL_30_Writing III_3",
    "000322_ESL_31_Reading III_3",
    "000322_ESL_32_Speaking III_2",
    "000322_ESL_40_Writing IIII_3",
    "000322_ESL_41_Reading IIII_3",
    "000322_ESL_42_Speaking IIII_2",
]

In [20]:
all_courses = math + english + history + statistics + economics + esl

all_courses

['000102_MAT_98_Pre-pre-Algebra_2',
 '000102_MAT_99_Pre-Algebra_2',
 '000102_MAT_100_Algebra_4',
 '000102_MAT_102_Linear Algebra_4',
 '000102_MAT_103_Linear Algebra II_4',
 '000102_MAT_110_Calculus_4',
 '000102_MAT_105_Geometry_4',
 '000102_MAT_107_Trigonometry_4',
 '000104_ENG_96_Pre-pre-pre-pre-College Composition_2',
 '000104_ENG_97_Pre-pre-pre-College Composition_2',
 '000104_ENG_98_Pre-pre-College Composition_2',
 '000104_ENG_99_Pre-College Composition_2',
 '000104_ENG_100_College Composition_4',
 '000104_ENG_101_College Composition II_4',
 '000104_ENG_102_American Literature_2',
 '000104_ENG_15_Reading I_2',
 '000104_ENG_25_Reading II_2',
 '000104_ENG_35_Reading III_2',
 '000104_ENG_45_Reading IIII_2',
 '000104_ENG_55_Reading V_2',
 '000156_HIS_100_World History I_3',
 '000156_HIS_101_World History II_3',
 '000156_HIS_110_Western Civilizations_4',
 '000156_HIS_115_Eastern Civilizations_5',
 '000156_HIS_120_Intro to Colonization_4',
 '000156_HIS_121_Colonization: Europe_5',
 '0001

In [21]:
# probabilities for each of the 50 courses
p = [
    .03,
    .02,
    .01,
    .005,
    .0005,
    .0005,
    .005,
    .009,
    .04,
    .03,
    .03,
    .03,
    .02,
    .01,
    .005,
    .03,
    .03,
    .03,
    .03,
    .03,
    .01,
    .01,
    .01,
    .01,
    .01,
    .01,
    .01,
    .02,
    .02,
    .02,
    .01,
    .015,
    .015,
    .015,
    .01,
    .01,
    .01,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
    .03,
]

In [22]:
courses = np.random.choice(a = all_courses, size = 10_000_000,)

In [23]:
courses_df = pd.DataFrame(courses)
courses_df.columns = ['course']

courses_df.head()

,course
0,000322_ESL_20_Writing II_3
1,000104_ENG_35_Reading III_2
2,000102_MAT_103_Linear Algebra II_4
3,000104_ENG_96_Pre-pre-pre-pre-College Composit...
4,004028_STA_77_Bio Statitics_3


In [24]:
courses_df = courses_df['course'].str.split("_", expand = True)

In [25]:
courses_df.columns = ['top_code', 'subject', 'course_number', 'course_name', 'units']
courses_df['units'] = courses_df['units'].astype(int)
courses_df.head()

,top_code,subject,course_number,course_name,units
0,000322,ESL,20,Writing II,3
1,000104,ENG,35,Reading III,2
2,000102,MAT,103,Linear Algebra II,4
3,000104,ENG,96,Pre-pre-pre-pre-College Composition,2
4,004028,STA,77,Bio Statitics,3


In [26]:
big_data['top_code'] = courses_df['top_code']
big_data['subject'] = courses_df['subject']
big_data['course_number'] = courses_df['course_number']
big_data['course_name'] = courses_df['course_name']
big_data['units'] = courses_df['units']

In [27]:
# delete unnecessary data as we go
del courses
del courses_df
del course_df
del course_example

In [28]:
big_data.head()

,ids,sex,age,first_time_status,intent,race,full_time,college,top_code,subject,course_number,course_name,units
0,471900,female,38,returning,AA,asian,full-time,Butte College,000322,ESL,20,Writing II,3
1,506657,female,35,returning,transfer,asian,full-time,San Diego Mesa College,000104,ENG,35,Reading III,2
2,780835,female,58,returning,credential,latino,part-time,Sierra College,000102,MAT,103,Linear Algebra II,4
3,258697,female,59,returning,transfer,white,full-time,De Anza College,000104,ENG,96,Pre-pre-pre-pre-College Composition,2
4,677933,female,45,returning,transfer,latino,full-time,San Jose City College,004028,STA,77,Bio Statitics,3


### Grades and Term (Year-Quarter)

In [29]:
grade_options = [
    'A+',
    'A',
    'A-',
    'B+',
    'B',
    'B-',
    'C+',
    'C',
    'C-',
    'D+',
    'D',
    'F',
]

distributions = [
    0.01,
    0.02,
    0.03,
    0.07,
    0.12,
    0.10,
    0.17,
    0.20,
    0.14,
    0.07,
    0.06,
    0.01,
]

grades = np.random.choice(a = grade_options, size = 10_000_000, p = distributions)

In [30]:
year = np.random.choice(a = ['2014', '2015', '2016'], size = 10_000_000,)
quarter = np.random.choice(a = ['Summer','Fall', 'Winter', 'Spring',], size = 10_000_000, p = [0.10, 0.30, 0.30, 0.30])

In [31]:
big_data['grades'] = grades
big_data['year'] = year
big_data['quarter'] = quarter

big_data.head()

,ids,sex,age,first_time_status,intent,race,full_time,college,top_code,subject,course_number,course_name,units,grades,year,quarter
0,471900,female,38,returning,AA,asian,full-time,Butte College,000322,ESL,20,Writing II,3,C,2015,Spring
1,506657,female,35,returning,transfer,asian,full-time,San Diego Mesa College,000104,ENG,35,Reading III,2,B-,2014,Winter
2,780835,female,58,returning,credential,latino,part-time,Sierra College,000102,MAT,103,Linear Algebra II,4,C+,2016,Spring
3,258697,female,59,returning,transfer,white,full-time,De Anza College,000104,ENG,96,Pre-pre-pre-pre-College Composition,2,C-,2014,Spring
4,677933,female,45,returning,transfer,latino,full-time,San Jose City College,004028,STA,77,Bio Statitics,3,B-,2015,Winter



Change to Categoricals: intent, race, year, quarter, subject.
Add Period column to do datetime analysis

In [32]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000000 entries, 0 to 9999999
Data columns (total 16 columns):
ids                  int64
sex                  object
age                  int32
first_time_status    object
intent               object
race                 object
full_time            object
college              object
top_code             object
subject              object
course_number        object
course_name          object
units                int32
grades               object
year                 object
quarter              object
dtypes: int32(2), int64(1), object(13)
memory usage: 1.2+ GB


__Memory usage 1.2+ GB__

To save memory we'll change: <br>
sex, first_time_status, intent, race, full_time, college, subject, grades to `categoricals` and change units to `int8`

In [33]:
cols_to_category = ['sex', 'first_time_status', 'intent', 'race', 'full_time', 'college', 'top_code', 'subject', 'course_number', 'grades']

for col in cols_to_category:
    big_data[col] = big_data[col].astype('category')

In [34]:
# Changing numerical columns to smaller ints because int32 and int64 are unnecessarily big data types for some columns.
big_data['units'] = big_data['units'].astype(np.int8)
big_data['ids'] = big_data['ids'].astype(np.uint32)
big_data['age'] = big_data['age'].astype(np.int8)
big_data['year'] = big_data['year'].astype(np.int16)

In [35]:
big_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000000 entries, 0 to 9999999
Data columns (total 16 columns):
ids                  uint32
sex                  category
age                  int8
first_time_status    category
intent               category
race                 category
full_time            category
college              category
top_code             category
subject              category
course_number        category
course_name          object
units                int8
grades               category
year                 int16
quarter              object
dtypes: category(10), int16(1), int8(2), object(2), uint32(1)
memory usage: 400.6+ MB


__Memory usage 400.6+ MB__

We'll still compress this to save it and keep in the repo.

In [36]:
import gzip

In [37]:
with gzip.open("../data/processed/fake_data.dta.gz", "wb") as file:
    big_data.to_stata(file, write_index = False,)

After compression the dataset `fake_data.dta.gz` ends up taking about __19MB__ of memory.